# LOADING DF TO GOOGLE BIGQUERY

In [1]:
import os
import pandas as pd
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas_gbq
import numpy as np

# extract xlsx file in dataframe
df = pd.read_excel('customer.xlsx')

# salve as csv
df_1=df.to_csv('customer.csv', index=False, encoding='utf-8')

In [2]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


In [3]:
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [4]:
path = 'C:/Users/vanes/OneDrive/Documentos/PROJETOS_PORTFOLIO/ETL/'

In [5]:
credentials = service_account.Credentials.from_service_account_file(path +'gbq.json', 
                                                                    scopes=["https://www.googleapis.com/auth/cloud-platform"])

In [6]:
df.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
dtype: object

In [7]:
df['CustomerID'].value_counts()

CustomerID
17841.0    7983
14911.0    5903
14096.0    5128
12748.0    4642
14606.0    2782
           ... 
15070.0       1
15753.0       1
17065.0       1
16881.0       1
16995.0       1
Name: count, Length: 4372, dtype: int64

In [8]:
df['InvoiceNo'] = df['InvoiceNo'].astype(str)
df['StockCode'] = df['StockCode'].astype(str)
df['Description'] = df['Description'].astype(str)
df['Quantity'] = df['Quantity'].astype('int64')
df['InvoiceDate']= pd.to_datetime(df['InvoiceDate'])
df['UnitPrice'] = df['UnitPrice'].astype('float64')
df['CustomerID'] = df['CustomerID'].astype(str)
df['Country'] = df['Country'].astype(str)

In [9]:


column_types = df.dtypes.astype(str).to_dict() # dictionary in format type: specify type

dicionario = {'object' : 'STRING', 'float64' : 'FLOAT64',  'int64' : 'INT64', 'datetime64[ns]' : 'DATETIME'} # dictionary to transform 

for column, type in column_types.items() :
    column_types[column] = dicionario[type] 

# Se quiser transformar uma coluna específica para um tipo específico que não foi mapeado:
# column_types[column] = 'STRING'

schema = [bigquery.SchemaField(name=column, field_type=tipo) for column, tipo in column_types.items()]

In [10]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path+"gbq.json"


In [11]:

from google.cloud import bigquery

# creating an BigQuery client instance 

client = bigquery.Client()

# table Id to load the data
table_id = "project_1.retail_df"

# Setting job loading 
job_config = bigquery.LoadJobConfig()

# loading Dataframe to BigQuery table
job = client.load_table_from_dataframe(df, table_id, job_config=job_config)

# waiting on the job conclusion
job.result()


LoadJob<project=etl-project-416319, location=US, id=eaafb13e-173a-4e95-a118-22b908da58a5>